## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Airai,TL,2021-12-12 22:29:03,-8.9266,125.4092,62.47,86,100,1.72,overcast clouds
1,1,Sinazongwe,ZM,2021-12-12 22:35:38,-17.2614,27.4618,80.92,45,20,3.13,few clouds
2,2,Freeport,US,2021-12-12 22:34:11,40.6576,-73.5832,47.17,48,1,5.01,clear sky
3,3,Samarai,PG,2021-12-12 22:35:38,-10.6167,150.6667,83.59,75,100,11.10,overcast clouds
4,4,Chaihe,CN,2021-12-12 22:35:39,44.7667,129.7000,5.70,72,33,10.69,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
1,1,Sinazongwe,ZM,2021-12-12 22:35:38,-17.2614,27.4618,80.92,45,20,3.13,few clouds
3,3,Samarai,PG,2021-12-12 22:35:38,-10.6167,150.6667,83.59,75,100,11.10,overcast clouds
6,6,Rikitea,PF,2021-12-12 22:31:21,-23.1203,-134.9692,77.65,76,87,8.63,light rain
11,11,Kapaa,US,2021-12-12 22:35:40,22.0752,-159.3190,80.92,72,75,1.99,broken clouds
13,13,Acarau,BR,2021-12-12 22:35:41,-2.8856,-40.1200,80.89,79,41,17.38,scattered clouds
17,17,Gorontalo,ID,2021-12-12 22:35:42,0.5412,123.0595,77.92,92,100,3.15,overcast clouds
19,19,East London,ZA,2021-12-12 22:35:20,-33.0153,27.9116,71.85,95,100,7.87,light rain
27,27,Ixtapa,MX,2021-12-12 22:35:45,20.7000,-105.2000,79.83,74,1,6.91,clear sky
29,29,Bubaque,GW,2021-12-12 22:35:46,11.2833,-15.8333,81.21,73,21,13.27,few clouds
31,31,Port Alfred,ZA,2021-12-12 22:33:03,-33.5906,26.8910,71.02,96,100,14.23,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID             238
City                238
Country             238
Date                238
Lat                 238
Lng                 238
Max Temp            238
Humidity            238
Cloudiness          238
Wind Speed          238
City Description    238
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
1,Sinazongwe,ZM,80.92,few clouds,-17.2614,27.4618,
3,Samarai,PG,83.59,overcast clouds,-10.6167,150.6667,
6,Rikitea,PF,77.65,light rain,-23.1203,-134.9692,
11,Kapaa,US,80.92,broken clouds,22.0752,-159.3190,
13,Acarau,BR,80.89,scattered clouds,-2.8856,-40.1200,
17,Gorontalo,ID,77.92,overcast clouds,0.5412,123.0595,
19,East London,ZA,71.85,light rain,-33.0153,27.9116,
27,Ixtapa,MX,79.83,clear sky,20.7000,-105.2000,
29,Bubaque,GW,81.21,few clouds,11.2833,-15.8333,
31,Port Alfred,ZA,71.02,overcast clouds,-33.5906,26.8910,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                238
Country             238
Max Temp            238
City Description    238
Lat                 238
Lng                 238
Hotel Name          238
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Vacation_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>dd>{City Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))